In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

## Definitions

1. [ ] Define input and output
 - Input: sequence of code tokens, where token to predict is <UNK>
 - Output: a list of predicted tokens, sorted in descending order of probability
1. [ ] Define evaluation metric
 - Should be rank-aware, e.g. MRR, MAP, NDCG.
 - The target token is split into subtokens and the overlap between the predicted token and the target token at subtoken level is evaluated. This can be done with an F1 score.
    - For example, if `transformSearchResponse` is the target token, its subtokens are `transform`, `search` and `response`. If the predicted token is `modifySearchResponse`, then the overlap is 2 subtokens out of 3.
    - [ ] Should we account for the order of the subtokens? Most probably, yes.
    ```
    Precision = TP / (TP + FP) = #overlapping-predicted / (#overlapping-predicted + #nonoverlapping-predicted)
    Recall = TP / (TP + FN) = #overlapping-predicted / (#overlapping-predicted + #nonoverlapping-required)
    F1 = 2 * P * R / (P + R)
    ```
 - The F1 approach is inspired by SQuAD and "Suggesting accurate method and class names" by Allamanis et al.

## Execution Tasks

1. [ ] Gather Data
1. [ ] Analyze Data
1. [ ] Implement an algorithm
1. [ ] Create an evaluation loop
1. [ ] Expose parameters of the algorithm
1. [ ] Make experiments
1. [ ] Document the experiments - Hypothesis, Data, Setup, Evaluation, Algorithm, Experiments, Conclusion, Further Steps



## Discussion
it isn't as simple as, jsut average some word embeddings
it is easy to average the embeddings if you need to predict a word
but i have to predict a an unknown amount of subtokens

well, why don't we try to predict subtoken at a time
but what type should it be?
well, we can split all tokens into subtokens and count the PoS occurrences
so that we know a few patterns upfront
and use these patterns to fill in the subtokens with averaged word embeddings
similar to the context we have, but filtered according to the PoS tag
this is an interesting idea, because we do suggestion at the subtoken level
i like this idea and we can try it

of course the other idea is to use source code embeddings directly
but they have to be learned on our source code base
and this is a separate problem
which involves learning a model
which is what we try to avoid with our simplistic baseline
we are just exploring ideas
a baseline without learning might be a failure
but the ideas we try and pick up during the design and experimentation are what is most valuable



In [14]:
def split_by_camel_case(token):
#     TODO: implement me
    return token

def get_subtokens(token):
    return split_by_camel_case(token)

def compute_f1(target_token, predicted_token):
    target_subtokens = get_subtokens(target_token)
    predicted_subtokens = get_subtokens(predicted_token)
    overlapping = Counter(target_subtokens) & Counter(predicted_subtokens)
    overlapping_count = sum(overlapping.values())
    
    precision = 1.0 * overlapping_count / len(predicted_subtokens)
    recall = 1.0 * overlapping_count / len(target_subtokens)
    f1 = (2.0 * precision * recall) / (precision + recall)
    return f1

In [15]:
compute_f1(['transform', 'search', 'response'], ['modify', 'search', 'response', 'data'])

0.5714285714285715